### Etapa 1.1: Conexão com o banco de dados e obtenção dos metadados
Essa etapa irá se conectar ao banco de dados PostgreSQL e obter os metadados de todas as tabelas.

In [ ]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

# Carregar variáveis do arquivo .env
load_dotenv()
# Função para conectar ao banco de dados
def connect_db():
    try:
            # Pegar as credenciais do arquivo .env
            db_host = os.getenv('DB_HOST')
            db_database = os.getenv('DB_DATABASE')
            db_user = os.getenv('DB_USER')
            db_password = os.getenv('DB_PASSWORD')
            db_port = os.getenv('DB_PORT')

            # Conectar ao banco de dados PostgreSQL usando as variáveis do .env
            conn = psycopg2.connect(
                host=db_host,
                database=db_database,
                user=db_user,
                password=db_password,
                port=db_port
            )
            print("Conexão com o banco de dados estabelecida com sucesso!")
            return conn
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None


# Função para obter os metadados de todas as tabelas
def get_all_table_metadata(conn):
    try:
        query = """
        SELECT table_name, column_name, data_type 
        FROM information_schema.columns 
        WHERE table_schema = 'public'
        ORDER BY table_name;
        """
        metadata_df = pd.read_sql_query(query, conn)
        print("Metadados obtidos com sucesso!")
        return metadata_df
    except Exception as e:
        print(f"Erro ao obter metadados: {e}")
        return None

# Testar a conexão e obtenção de metadados
conn = connect_db()

if conn:
    metadata_df = get_all_table_metadata(conn)
    if metadata_df is not None:
        print(metadata_df.head())  # Mostrando uma amostra dos metadados obtidos
    else:
        print("Erro ao carregar metadados.")
else:
    print("Erro na conexão com o banco de dados.")

In [ ]:
metadata_df

### USANDO SLQALCHEMY
#### Etapa 1.2 : Conexão com o banco de dados e obtenção dos metadados
Essa etapa irá se conectar ao banco de dados PostgreSQL e obter os metadados de todas as tabelas.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Carregar variáveis do arquivo .env
load_dotenv()

# Função para conectar ao banco de dados usando SQLAlchemy
def connect_db_sqlalchemy():
    try:
        # Pegar as credenciais do arquivo .env
        db_host = os.getenv('DB_HOST')
        db_database = os.getenv('DB_DATABASE')
        db_user = os.getenv('DB_USER')
        db_password = os.getenv('DB_PASSWORD')
        db_port = os.getenv('DB_PORT')

        # Criar a URL de conexão no formato aceito pelo SQLAlchemy
        db_url = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}'
        
        # Criar o engine de conexão usando SQLAlchemy
        engine = create_engine(db_url)
        print("Conexão com o banco de dados estabelecida com sucesso via SQLAlchemy!")
        return engine
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados com SQLAlchemy: {e}")
        return None

# Função para obter os metadados de todas as tabelas usando SQLAlchemy
def get_all_table_metadata_sqlalchemy(engine):
    try:
        query = """
        SELECT table_name, column_name, data_type 
        FROM information_schema.columns 
        WHERE table_schema = 'public'
        ORDER BY table_name;
        """
        metadata_df = pd.read_sql_query(query, engine)
        print("Metadados obtidos com sucesso via SQLAlchemy!")
        return metadata_df
    except Exception as e:
        print(f"Erro ao obter metadados via SQLAlchemy: {e}")
        return None

# Testar a conexão e obtenção de metadados com SQLAlchemy
engine = connect_db_sqlalchemy()

if engine:
    metadata_df = get_all_table_metadata_sqlalchemy(engine)
    if metadata_df is not None:
        print(metadata_df.head())  # Mostrando uma amostra dos metadados obtidos
    else:
        print("Erro ao carregar metadados.")
else:
    print("Erro na conexão com o banco de dados via SQLAlchemy.")


In [ ]:
metadata_df

### Etapa 2.1: Gerar o contexto a partir dos metadados
Essa etapa transforma os metadados em um contexto que o LLM pode usar para entender a estrutura do banco de dados.

In [ ]:
# Função que cria um contexto a partir dos metadados de todas as tabelas
def create_context_from_metadata(metadata_df):
    try:
        context = "O banco de dados possui as seguintes tabelas e colunas:\n"
        
        # Agrupar metadados por tabela
        tables = metadata_df.groupby('table_name')
        
        # Para cada tabela, listar as colunas e seus tipos
        for table_name, table_data in tables:
            context += f"\nTabela '{table_name}' possui as seguintes colunas:\n"
            for index, row in table_data.iterrows():
                context += f"- {row['column_name']} ({row['data_type']})\n"
        
        print("Contexto gerado com sucesso!")
        return context
    except Exception as e:
        print(f"Erro ao criar o contexto: {e}")
        return None

# Gerar o contexto se os metadados foram carregados
if metadata_df is not None:
    contexto_banco = create_context_from_metadata(metadata_df)
    print(contexto_banco[:500])  # Mostrar parte do contexto gerado (primeiros 500 caracteres)
else:
    print("Erro: os metadados não estão disponíveis para gerar o contexto.")


### Etapa 2.Geracao de contexto

Consulta aprimorada: Estamos agora capturando mais informações sobre as colunas, incluindo:

1. table_name: Nome da tabela.
2. column_name: Nome da coluna.
3. data_type: Tipo de dado da coluna (ex. varchar, integer).
4. is_nullable: Informa se a coluna pode ser nula (YES ou NO).
5. character_maximum_length: Limite de caracteres (se for do tipo varchar ou outros tipos de texto).
6. Organização dos resultados: O resultado é organizado em um dataframe pandas, com as informações sobre as tabelas e colunas do banco de dados.
7. Contagem de tabelas: O código também imprime a quantidade de tabelas únicas no banco de dados, utilizando o método nunique() do pandas para contar as tabelas.
8. Debug visual: exibe as 10 primeiras linhas dos metadados para que você possa verificar visualmente se os dados foram carregados corretamente.


---MELHORIAS ---
1. TRATAR ERROS DE CONSULTA
2. MELHORAR O PROMPT PARA TRAZER MAIOR ASSERTIVIDADE NAS CONSULTAS
3.

In [ ]:
# Função para obter os metadados de todas as tabelas usando SQLAlchemy
def get_all_table_metadata_sqlalchemy(engine):
    try:
        # Consulta para buscar nome da tabela, nome da coluna e tipo de dado
        query = """
        SELECT table_name, column_name, data_type, is_nullable, character_maximum_length
        FROM information_schema.columns 
        WHERE table_schema = 'public'
        ORDER BY table_name, ordinal_position;
        """
        
        # Executar a consulta e obter o resultado em um dataframe
        metadata_df = pd.read_sql_query(query, engine)
        print("Metadados obtidos com sucesso via SQLAlchemy!")
        
        # Mostrar algumas linhas para verificar se está correto
        print(metadata_df.head())
        
        # Exibir uma visão geral do número de tabelas
        table_count = metadata_df['table_name'].nunique()
        print(f"O banco de dados contém {table_count} tabelas.")

        return metadata_df
    except Exception as e:
        print(f"Erro ao obter metadados via SQLAlchemy: {e}")
        return None

# Testar a função de obtenção de metadados com SQLAlchemy
if engine:
    metadata_df = get_all_table_metadata_sqlalchemy(engine)
    if metadata_df is not None:
        # Visualizar o dataframe com os metadados
        print(metadata_df.head(10))  # Mostrar as 10 primeiras linhas
    else:
        print("Erro ao carregar metadados.")
else:
    print("Erro na conexão com o banco de dados via SQLAlchemy.")


In [ ]:
contexto_banco

### Etapa 3.1: Gerar consulta SQL a partir da pergunta em linguagem natural
Agora, vamos integrar o LangChain e o OpenAI para transformar a pergunta do usuário em uma query SQL

In [ ]:
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate

# Configuração do OpenAI e LangChain
def generate_sql_from_question(pergunta, contexto):
    try:
        prompt_template = """
        Você é um assistente que entende a estrutura de um banco de dados. Aqui estão os metadados do banco:

        {contexto_tabela}

        Baseado nesses metadados, escreva uma consulta SQL para responder à seguinte pergunta:

        {pergunta}

        SQL:
        """
        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["contexto_tabela", "pergunta"]
        )

        # Configurar a API key da OpenAI
        openai.api_key = "sua_chave_openai"  # Substitua com sua chave de API OpenAI
        llm = OpenAI(model="text-davinci-003")  # Usando o modelo da OpenAI

        # Criar a cadeia de LangChain para gerar a query SQL
        chain = LLMChain(llm=llm, prompt=prompt)

        # Gerar a query SQL
        sql_query = chain.run(contexto_tabela=contexto, pergunta=pergunta)
        print("Query SQL gerada com sucesso!")
        return sql_query
    except Exception as e:
        print(f"Erro ao gerar a query SQL: {e}")
        return None

# Exemplo de pergunta do usuário
pergunta_usuario = "Quantas pessoas a unidade de saúde Maria das Graças atendeu hoje?"

if contexto_banco:
    sql_query_gerada = generate_sql_from_question(pergunta_usuario, contexto_banco)
    if sql_query_gerada:
        print(f"Query gerada: {sql_query_gerada}")
    else:
        print("Erro ao gerar a query.")
else:
    print("Contexto do banco de dados não disponível.")


### 3.2 Código para Gerar Consultas SQL a Partir de Perguntas em Linguagem Natural
Abaixo está o código para a integração com o LangChain e o OpenAI, usando a chave da API que será carregada do arquivo .env. Este código cria um prompt dinâmico para gerar a consulta SQL com base nos metadados que obtivemos na etapa anterior.

Explicação do Código:
1. Carregamento da chave OpenAI a partir do .env:
2. A chave da API da OpenAI é carregada do arquivo .env usando o método os.getenv(). Isso evita que você coloque a chave diretamente no código, garantindo segurança.
3. PromptTemplate para LangChain:
4. Criamos um template de prompt para LangChain, que inclui:
5. Os metadados do banco (contexto_tabela), que ajudam o modelo a entender a estrutura do banco.
6. A pergunta em linguagem natural do usuário (pergunta).
7. O prompt é passado para o modelo OpenAI via LangChain, que usa o modelo padrao (que da data de hoje é o 3.5 turbo),  para gerar a consulta SQL com base nos metadados e na pergunta.
8. Geração da Consulta SQL:
9. O resultado gerado pelo modelo é a consulta SQL, que será exibida na saída.
Verificação de Erros:
10. O código contém verificações de erros para garantir que a chave da API foi carregada corretamente, o prompt foi gerado adequadamente e a consulta SQL foi retornada com sucesso.


------MELHORIAS---------------

DETECTOR DE ERROS DE QUERY
RELACAO ENTRE AS TABELAS
TRATAMENTO DE ASPAS


In [ ]:
# Importações necessárias
import openai
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# Carregar as variáveis do .env, incluindo a chave da API do OpenAI
load_dotenv()

# Função para carregar a chave da API OpenAI do .env
def get_openai_api_key():
    try:
        openai_api_key = os.getenv('OPENAI_API_KEY')
        if not openai_api_key:
            raise ValueError("A chave da API OpenAI não foi encontrada no arquivo .env.")
        print("Chave da API OpenAI carregada com sucesso.")
        return openai_api_key
    except Exception as e:
        print(f"Erro ao carregar a chave da API: {e}")
        return None

# Função para gerar uma consulta SQL a partir de uma pergunta usando LangChain e OpenAI
def generate_sql_from_question(pergunta, contexto):
    try:
        # Carregar a chave da API OpenAI do .env
        openai_api_key = get_openai_api_key()
        if not openai_api_key:
            return None

        # Configurar a API key para o OpenAI
        openai.api_key = openai_api_key

        # Definir o template do prompt que será passado para o LLM
        prompt_template = """
        Você é um assistente de banco de dados que entende a estrutura do banco de dados PostgresSQl. Aqui estão os metadados do banco:

        {contexto_tabela}

        Baseado nesses metadados, escreva uma consulta SQL para responder à seguinte pergunta:

        {pergunta}

        SQL:
        """

        # Criar o template do LangChain
        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["contexto_tabela", "pergunta"]
        )

        # Inicializar o modelo OpenAI para LangChain
        llm = OpenAI()

        # Configurar a cadeia de LangChain para gerar a query SQL
        chain = LLMChain(llm=llm, prompt=prompt)

        # Gerar a consulta SQL com base na pergunta do usuário e nos metadados
        sql_query = chain.run(contexto_tabela=contexto, pergunta=pergunta)
        print("Consulta SQL gerada com sucesso!")
        return sql_query
    except Exception as e:
        print(f"Erro ao gerar a consulta SQL: {e}")
        return None

# Exemplo de pergunta do usuário e contexto dos metadados gerados anteriormente
pergunta_usuario = "quantas pessoal foram atendida em 2018?"

# Gerar uma amostra de contexto baseado nos metadados obtidos (você pode expandir isso conforme necessário)
if metadata_df is not None:
    contexto_banco = metadata_df.to_string(index=False)

    # Gerar a consulta SQL usando a pergunta do usuário e o contexto dos metadados
    sql_query_gerada = generate_sql_from_question(pergunta_usuario, contexto_banco)

    # Mostrar a consulta SQL gerada
    if sql_query_gerada:
        print(f"Consulta SQL gerada:\n{sql_query_gerada}")
    else:
        print("Erro ao gerar a consulta SQL.")
else:
    print("Os metadados do banco de dados não estão disponíveis.")


### Etapa 4.1 : Executar a consulta SQL gerada

Nesta etapa, vamos executar a consulta SQL gerada e retornar os resultados como um dataframe.

In [ ]:
# Função para executar a query no banco de dados
def execute_query(conn, query):
    try:
        df_result = pd.read_sql_query(query, conn)
        print("Query executada com sucesso!")
        return df_result
    except Exception as e:
        print(f"Erro ao executar a query: {e}")
        return None

# Se a query foi gerada com sucesso, execute-a no banco de dados
if sql_query_gerada:
    df_resultado = execute_query(conn, sql_query_gerada)
    if df_resultado is not None:
        print(df_resultado.head())  # Mostrar uma amostra dos resultados
    else:
        print("Erro ao executar a query.")
else:
    print("Query não disponível para execução.")


### 4.2 Atualização do Código para Executar a Consulta SQL
Abaixo está o código para a execução da consulta SQL gerada, utilizando o SQLAlchemy para conectar ao banco e o pandas para manipular os resultados.

In [ ]:
# Função para executar a consulta SQL gerada no banco de dados
def execute_query(engine, query):
    try:
        # Executar a consulta SQL e retornar o resultado em um dataframe pandas
        df_result = pd.read_sql_query(query, engine)
        print("Consulta SQL executada com sucesso!")
        return df_result
    except Exception as e:
        print(f"Erro ao executar a consulta SQL: {e}")
        return None

# Verificar se a consulta SQL foi gerada corretamente
if sql_query_gerada:
    # Executar a consulta SQL gerada no banco de dados
    df_resultado = execute_query(engine, sql_query_gerada)

    # Exibir os resultados obtidos, se houver algum
    if df_resultado is not None:
        print("Resultado da consulta:")
        print(df_resultado.head())  # Mostra as primeiras linhas do resultado
    else:
        print("Nenhum resultado encontrado ou erro na execução da consulta.")
else:
    print("Nenhuma consulta SQL foi gerada.")


### Etapa 5.1 : Interpretar o resultado e devolver resposta em linguagem natural
Aqui o agente interpreta os resultados e retorna a resposta em linguagem natural para o usuário.

In [ ]:
# Função para interpretar o resultado da query e devolver resposta em linguagem natural
def interpret_result(df_resultado):
    try:
        if not df_resultado.empty:
            total = df_resultado.iloc[0, 0]
            return f"Foram atendidas {total} pessoas na unidade."
        return "Nenhum dado encontrado."
    except Exception as e:
        print(f"Erro ao interpretar o resultado: {e}")
        return "Erro na interpretação dos dados."

# Se o resultado foi obtido, interpretar e devolver a resposta
if df_resultado is not None:
    resposta = interpret_result(df_resultado)
    print(f"Resposta em linguagem natural: {resposta}")
else:
    print("Não há resultados para interpretar.")


### 5.3 minha forma de fazer

In [ ]:
# Função para gerar o prompt de interpretação de resultados
def interpret_result_with_template(df_resultado, pergunta_usuario, sql_query_gerada):
    try:
        # Transformar o dataframe em string para que o modelo possa interpretar
        result_str = df_resultado.to_string(index=False)
        
        # Criar um template de prompt para LangChain interpretar os resultados
        prompt_template = """
        Você é um assistente de banco de dados. O usuário fez a seguinte pergunta:

        Pergunta: {pergunta}

        Você gerou a seguinte consulta SQL com base nos metadados do banco de dados:

        SQL: {sql_query}

        A consulta SQL retornou os seguintes dados:

        {resultado}

        Com base nisso, explique o resultado da consulta de forma clara e amigável e direta para o usuário.
        """

        # Criar o template do LangChain para processar o prompt
        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["pergunta", "sql_query", "resultado"]
        )

        # Inicializar o modelo OpenAI para LangChain
        llm = OpenAI()
        
        # Criar a cadeia LangChain para gerar a interpretação da resposta
        chain = LLMChain(llm=llm, prompt=prompt)

        # Gerar a resposta interpretada em linguagem natural
        resposta_interpretada = chain.run(pergunta=pergunta_usuario, sql_query=sql_query_gerada, resultado=result_str)
        return resposta_interpretada
    except Exception as e:
        print(f"Erro ao interpretar o resultado: {e}")
        return "Houve um erro ao interpretar os resultados."

# Exemplo de execução da função de interpretação de resultados
if df_resultado is not None and sql_query_gerada:
    resposta_interpretada = interpret_result_with_template(df_resultado, pergunta_usuario, sql_query_gerada)
    print(f"Resposta interpretada em linguagem natural: {resposta_interpretada}")
else:
    print("Não há resultados ou consulta SQL para interpretar.")
